以前曾经借助翻译软件看个大概，不对之处请多指正：\
Wthor文件分3种：\
对局库：WTH_2002.WTB，每年一期\
棋手库：WTHOR.JOU，记录各棋手的名字\
比赛库：WTHOR.TRN，记录各种比赛的名称\
3种文件的结构都很相似：16字节文件头+N条记录\
文件头结构：\
1字节 文件建立年份的百位部分，如20表示20**年\
1字节 文件建立年份的低2位\
1字节 文件建立的月份\
1字节 文件建立的日期\
4字节 N1，文件包含的记录数（这个是对局库用的）\
2字节 N2，文件包含的记录数（这个是棋手库和比赛库用的）\
2字节 对局年份\
1字节 P1，棋盘大小，0或8表示8*8\
1字节 P2，游戏类型，1表示solitaires游戏，0表示其他（包括黑白棋）\
1字节 P3，最佳终局计算的搜索深度，一般为22\
1字节 保留\
对局库记录结构，对于8*8黑白棋，每条记录68字节\
2字节 比赛名称，这里只是个序号，具体名称要查比赛库中的相应记录\
2字节 黑方棋手名字，这里只是个序号，具体名字要查棋手库\
2字节 白方棋手名字\
1字节 对局结束时黑方的棋子数（多余空格归胜方）\
1字节 黑方理论最佳得子，即在剩余P3个空格时，进行最佳终局计算，黑方的得子数\
60字节 棋步序列，每步棋一个字节，记为行号*10+列号，如A1=11，A2=21。欠行的步不作记录，只是将最后多余字节填0\
棋手库记录结构\
每条记录20字节，字符串类型（以 结尾），记录棋手名字。对于程序，一般用括号加注程序作者，如\
Thor(quin/becquet)\
比赛库记录结构\
每条记录26字节，字符串类型，记录比赛名称。\

In [2]:
import json
file_name = "rewth/WTH_{}.wtb"
total = 0
for file_year in range(2001,2016):
    with open(file_name.format(file_year),"rb") as file_:
        lines = file_.read()
        
    # print('year',int.from_bytes(lines[0:1], byteorder='little', signed=False))
    # print('year',int.from_bytes(lines[1:2], byteorder='little', signed=True))
    # print('month',int.from_bytes(lines[2:3], byteorder='little', signed=True))
    # print('day',int.from_bytes(lines[3:4], byteorder='little', signed=True))
    print(file_year,'record_num',int.from_bytes(lines[4:8], byteorder='little', signed=True))
    # print('0|',int.from_bytes(lines[8:10], byteorder='little', signed=True))
    # print('year',int.from_bytes(lines[10:12], byteorder='little', signed=True))
    # print('0or8|',int.from_bytes(lines[12:13], byteorder='little', signed=True))
    # print('0|',int.from_bytes(lines[13:14], byteorder='little', signed=True))
    # print('deep',int.from_bytes(lines[14:15], byteorder='little', signed=True))
    # print('reserve',int.from_bytes(lines[15:16], byteorder='little', signed=True))
    record_num=int.from_bytes(lines[4:8], byteorder='little', signed=True)
    total += record_num
    dataset = []
    for record_idx in range(record_num):
        start_idx = 16+record_idx*68
        name = lines[start_idx:start_idx+6]
        int.from_bytes(lines[start_idx+7:start_idx+8], byteorder='little', signed=True)
        # print(lines[start_idx+6:start_idx+7])
        black_chess_num = int.from_bytes(lines[start_idx+6:start_idx+7], byteorder='little', signed=True)
        if black_chess_num>32:
            who_win = 0
        elif black_chess_num<32:
            who_win = 1
        else:  
            who_win = 2
        black_chess_num_theory = int.from_bytes(lines[start_idx+7:start_idx+8], byteorder='little', signed=True)
        moves =[int.from_bytes(lines[start_idx+8+i:start_idx+8+i+1], byteorder='little', signed=True) for i in range(60)]
        # moves = lines[start_idx+8:start_idx+68]
        dataset.append({'black_chess_num':black_chess_num,
                        'black_chess_num_theory':black_chess_num_theory,
                        'who_win':who_win,
                        'moves':moves})
    output_filename = 'rewth/{}.json'                  
    with open(output_filename.format(file_year),'w') as file_obj:
        json.dump(dataset,file_obj)
# print(dataset)
print(total)


2001 record_num 5575
2002 record_num 5423
2003 record_num 3858
2004 record_num 9113
2005 record_num 4199
2006 record_num 2942
2007 record_num 2478
2008 record_num 2232
2009 record_num 4348
2010 record_num 2172
2011 record_num 1891
2012 record_num 2208
2013 record_num 2396
2014 record_num 1817
2015 record_num 1789
52441


In [38]:
print(total)

52441


In [3]:
import json


record = []
for file_year in range(2001,2016):
    with open(output_filename.format(file_year),'r') as file_obj:
        record += json.load(file_obj)    
print(len(record)) 
with open('rewth/2001-2015.json','w') as file_obj:
    json.dump(record,file_obj)
    


52441


In [4]:
with open('rewth/2001-2015.json','r') as file_obj:
    record = json.load(file_obj)
dataset = { 'black_chess_num':[],
            'black_chess_num_theory':[],
            'who_win':[],
            'moves':[]
            }
print(len(record))
for i in record:
    dataset['black_chess_num'].append(i["black_chess_num"])
    dataset['black_chess_num_theory'].append(i["black_chess_num_theory"])
    dataset['who_win'].append(i['who_win'])
    dataset['moves'].append(i['moves'])
print(len(dataset['who_win']))
with open('rewth/2001-2015_dataset.json','w') as file_obj:
    json.dump(dataset,file_obj)

52441
52441


In [5]:
import json
with open('rewth/2001-2015_dataset.json','r') as file_obj:
    data = json.load(file_obj)
count = 0
for idx,i in enumerate(data['moves']):
    if 0 in i:
        # print(i,data['black_chess_num'][idx])
        count += 1
print(data['moves'][0])
print(len(data['moves']))
print(count)

[56, 64, 53, 46, 35, 63, 34, 66, 65, 74, 37, 43, 57, 33, 76, 24, 75, 26, 83, 36, 73, 38, 25, 16, 14, 15, 17, 47, 13, 68, 48, 58, 52, 28, 67, 23, 12, 61, 32, 42, 31, 86, 51, 41, 27, 84, 85, 82, 71, 18, 72, 11, 21, 22, 62, 81, 77, 78, 88, 87]
52441
2340


In [6]:
def two2_(idx):
    if idx==1:
        return '1'
    elif idx==2:
        return '.'
    elif idx==0:
        return '0'

In [9]:
ffrom src.utils import *
import json
from tqdm import tqdm
with open('rewth/2001-2015_dataset.json','r') as file_obj:
    data = json.load(file_obj)
init_board = [
            2,2,2,2,2,2,2,2,
            2,2,2,2,2,2,2,2,
            2,2,2,2,2,2,2,2,
            2,2,2,1,0,2,2,2,
            2,2,2,0,1,2,2,2,
            2,2,2,2,2,2,2,2,
            2,2,2,2,2,2,2,2,
            2,2,2,2,2,2,2,2
        ]  
dataset = {'board':[],'next_step_id':[],'turn':[],'black_chess_num':[]}
for idx,chessbook in enumerate(tqdm(data['moves'])):

    # chessbook = data['moves'][1000]
    black_chess_num = data['black_chess_num'][idx]
    turn = 0
    now_board = init_board
    for idx,step in enumerate(chessbook):
        # if step == 11:
        #     print(11)
        if step == 0:
            break
        x = step%10 -1
        y = step//10 -1
        id = xy2id((x,y))
        
        # n_step = chessbook[idx+1]
        # if n_step==0:
        #     break
        # n_x = n_step%10 -1
        # n_y = n_step//10 -1
        # n_id = xy2id((n_x,n_y))
        next_pos = ask_next_pos(now_board,turn,need_id = True)
        if len(next_pos) == 0:
            turn = 0 if turn==1 else 1
            next_pos = ask_next_pos(now_board,turn,need_id = True)
            assert len(next_pos) !=0 , 'turn wrong'
            
        dataset['board'].append(now_board.copy())
        dataset['next_step_id'].append(id)
        dataset['turn'].append(turn)
        dataset['black_chess_num'].append(black_chess_num) 
          
        next_board = get_next_step_board(now_board,turn,id)
        now_board = next_board.copy()
        # next_board = list(map(two2_,next_board))
        # print(next_board[0:8])
        # print(next_board[8:16])
        # print(next_board[16:24])
        # print(next_board[24:32])
        # print(next_board[32:40])
        # print(next_board[40:48])
        # print(next_board[48:56])
        # print(next_board[56:64])
        # print('\n')
        if len(next_pos) >0:
            turn = 0 if turn==1 else 1
with open('rewth/2001-2015_dataset_board.json','w') as file_obj:
    json.dump(dataset,file_obj)

100%|██████████| 52441/52441 [07:49<00:00, 111.77it/s]


In [3]:
import json
with open('rewth/2001-2015_dataset_board.json','r') as file_obj:
    data = json.load(file_obj)